In [1]:
import pandas as pd
import sqlite3
import os
import numpy as np
from IPython.display import display

# --- 1. 設定 ---
DB_PATH = "data/processed/s2orc_filtered.db"
# 読み込むファイル
BASE_EVAL_PAPERS_FILE = "data/datapapers/sampled/evaluation_data_papers_50.csv"
TRAINING_PAPERS_FILE = "data/datapapers/sampled/training_data_papers_50.csv"

# ▼▼▼ 修正点: 新しい出力ファイル名 ▼▼▼
FINAL_EVAL_PAPERS_FILE = "data/datapapers/sampled/evaluation_data_papers_50_v2.csv"

TARGET_EVAL_SIZE = 50 # 最終的な目標件数

print("--- Rebuilding Evaluation Set to Target Size ---")
print(f"Input file: {BASE_EVAL_PAPERS_FILE}")
print(f"Output file: {FINAL_EVAL_PAPERS_FILE}")

--- Rebuilding Evaluation Set to Target Size ---
Input file: data/datapapers/sampled/evaluation_data_papers_50.csv
Output file: data/datapapers/sampled/evaluation_data_papers_50_v2.csv


In [2]:
# --- ステップ1: 現状の評価セットをクリーンアップ ---
print(f"\nLoading current evaluation set: {BASE_EVAL_PAPERS_FILE}")
try:
    df_eval_current = pd.read_csv(BASE_EVAL_PAPERS_FILE)
    current_eval_dois = tuple(df_eval_current['cited_datapaper_doi'].unique())
    print(f"Current file has {len(df_eval_current)} entries.")
except Exception as e:
    print(f"Error loading CSV: {e}")
    raise

# DBに接続し、ペア作成可能（Human: Used >= 2）なDOIを特定
valid_eval_dois = []
invalid_eval_dois = []
with sqlite3.connect(DB_PATH) as conn:
    for doi in current_eval_dois:
        # 評価スクリプト(22b)のセル4のロジックと合わせる
        query_gt = "SELECT COUNT(citing_doi) FROM positive_candidates WHERE cited_datapaper_doi = ? AND human_annotation_status = 1"
        count = conn.execute(query_gt, (doi,)).fetchone()[0]
        
        if count >= 2: # 正解が2件以上（クエリ1件 + 正解1件以上）あるもの
            valid_eval_dois.append(doi)
        else:
            invalid_eval_dois.append(doi)

print(f"\nFound {len(valid_eval_dois)} valid papers (Human Used >= 2).")
print(f"Found {len(invalid_eval_dois)} invalid papers (Human Used < 2) to be removed:")
print(invalid_eval_dois)

# 有効な論文（46件のはず）のDataFrameを作成
df_eval_valid = df_eval_current[df_eval_current['cited_datapaper_doi'].isin(valid_eval_dois)].copy()


Loading current evaluation set: data/datapapers/sampled/evaluation_data_papers_50.csv
Current file has 50 entries.

Found 46 valid papers (Human Used >= 2).
Found 4 invalid papers (Human Used < 2) to be removed:
['10.5194/ESSD-14-1917-2022', '10.1111/GEB.13179', '10.1038/S41597-019-0288-Y', '10.1016/J.DIB.2019.104313']


In [3]:
# --- ステップ2: 不足分の候補を選定 ---

# ▼▼▼ 修正点1: tabulateをインストール ▼▼▼
%pip install tabulate

n_needed = TARGET_EVAL_SIZE - len(df_eval_valid)

if n_needed <= 0:
    print(f"\nEvaluation set already has {len(df_eval_valid)} valid papers. No additions needed.")
    df_final_eval_set = df_eval_valid
else:
    print(f"\nNeed to add {n_needed} new data papers.")
    
    df_train = pd.read_csv(TRAINING_PAPERS_FILE)
    train_dois = set(df_train['cited_datapaper_doi'])
    used_dois = set(valid_eval_dois).union(train_dois)
    print(f"Total DOIs already in use (Train + Valid Eval): {len(used_dois)}")

    try:
        with sqlite3.connect(DB_PATH) as conn:
            query = """
                SELECT
                    cited_datapaper_doi,
                    COUNT(citing_doi) AS llm_used_count
                FROM
                    positive_candidates
                WHERE
                    llm_annotation_status = 1
                GROUP BY
                    cited_datapaper_doi
                HAVING
                    COUNT(citing_doi) >= 2;
            """
            df_all_eligible = pd.read_sql_query(query, conn)
            
            df_available = df_all_eligible[~df_all_eligible['cited_datapaper_doi'].isin(used_dois)]
            print(f"Available pool size (unused papers): {len(df_available)}")
            
            if len(df_available) < n_needed:
                print(f"❌ Error: Not enough available papers in the pool ({len(df_available)}) to find {n_needed} replacements.")
            else:
                df_replacements = df_available.sort_values(by='llm_used_count', ascending=False).head(n_needed)
                
                print("\n" + "="*50)
                print(f"✅ Selected {n_needed} new data papers (Top {n_needed} by LLM 'Used' count):")
                
                # ▼▼▼ 修正点2: 表示エラーを握りつぶす ▼▼▼
                try:
                    print(df_replacements.to_markdown(index=False))
                except ImportError:
                    print("(Could not print markdown, showing simple list):")
                    print(df_replacements)
                # ▲▲▲ --------------------------------- ▲▲▲
                    
                print("="*50)
                
                df_final_eval_set = pd.concat([df_eval_valid, df_replacements], ignore_index=True)

    except Exception as e:
        print(f"💥 An error occurred during selection: {e}")

Note: you may need to restart the kernel to use updated packages.

Need to add 4 new data papers.
Total DOIs already in use (Train + Valid Eval): 196
Available pool size (unused papers): 525

✅ Selected 4 new data papers (Top 4 by LLM 'Used' count):
| cited_datapaper_doi       |   llm_used_count |
|:--------------------------|-----------------:|
| 10.5194/ESSD-14-1917-2022 |               25 |
| 10.1016/J.DIB.2016.05.025 |                8 |
| 10.1016/J.DIB.2018.11.111 |                5 |
| 10.1038/S41597-019-0288-Y |                4 |


In [4]:
# --- ステップ3: DBの一括更新とCSVの「新規」保存 ---
if 'df_replacements' in locals() and not df_replacements.empty:
    
    new_dois_to_update = tuple(df_replacements['cited_datapaper_doi'].unique())
    
    print(f"\nUpdating DB for {len(new_dois_to_update)} new papers...")
    print("Setting 'human_annotation_status = 1' for all 'llm_annotation_status = 1' candidates...")
    
    try:
        with sqlite3.connect(DB_PATH) as conn:
            cursor = conn.cursor()
            
            # 1. 更新対象の件数を事前にカウント
            placeholders = ','.join('?' for _ in new_dois_to_update)
            count_query = f"""
                SELECT COUNT(*)
                FROM positive_candidates
                WHERE cited_datapaper_doi IN ({placeholders})
                  AND llm_annotation_status = 1
                  AND human_annotation_status = 0
            """
            count_before = cursor.execute(count_query, new_dois_to_update).fetchone()[0]
            print(f"Found {count_before:,} candidate rows to mark as 'Human: Used'.")
            
            # 2. 一括更新（UPDATE）の実行
            update_query = f"""
                UPDATE positive_candidates
                SET human_annotation_status = 1
                WHERE cited_datapaper_doi IN ({placeholders})
                  AND llm_annotation_status = 1
                  AND human_annotation_status = 0
            """
            cursor.execute(update_query, new_dois_to_update)
            updated_rows = cursor.rowcount
            conn.commit()
            print(f"✅ DB Update Complete. {updated_rows:,} rows affected.")

    except Exception as e:
        print(f"💥 An error occurred during DB update: {e}")

# --- ▼▼▼ 修正点: 「上書き」ではなく「新しいファイル」に保存 ---
if 'df_final_eval_set' in locals():
    print(f"\nSaving final evaluation set with {len(df_final_eval_set)} papers to NEW FILE: {FINAL_EVAL_PAPERS_FILE}...")
    # 元のCSVを上書きせず、新しいファイル名で保存
    df_final_eval_set.to_csv(FINAL_EVAL_PAPERS_FILE, index=False)
    print("✅ New CSV file created successfully.")
    print("\n--- Process Complete ---")
    print(f"元のファイル '{BASE_EVAL_PAPERS_FILE}' は変更されていません。")
    print(f"今後の評価スクリプトでは、'{FINAL_EVAL_PAPERS_FILE}' を参照するようにしてください。")
else:
    print("No changes made.")


Updating DB for 4 new papers...
Setting 'human_annotation_status = 1' for all 'llm_annotation_status = 1' candidates...
Found 0 candidate rows to mark as 'Human: Used'.
✅ DB Update Complete. 0 rows affected.

Saving final evaluation set with 50 papers to NEW FILE: data/datapapers/sampled/evaluation_data_papers_50_v2.csv...
✅ New CSV file created successfully.

--- Process Complete ---
元のファイル 'data/datapapers/sampled/evaluation_data_papers_50.csv' は変更されていません。
今後の評価スクリプトでは、'data/datapapers/sampled/evaluation_data_papers_50_v2.csv' を参照するようにしてください。
